In [147]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## MNIST Data Set
MNIST Data Set contains data for 28x28 pixel gray scale images of handwritten numbers from 0-9. Specifically, the data represents 784 (28*28) pixels per image that can take on values from 0-255. Zero being the color black and 255 being white. 

In [161]:
MNIST_train = pd.read_csv("mnist_train.csv")
MNIST_train = pd.concat([MNIST_train,pd.get_dummies(MNIST_train.label,prefix = "label")],axis=1)
MNIST_train.drop(columns=['label'], axis =1, inplace = True)
MNIST_test = pd.read_csv("mnist_test.csv")
MNIST_test = pd.concat([MNIST_test,pd.get_dummies(MNIST_test.label,prefix = "label")],axis=1)
MNIST_test.drop(columns=['label'], axis =1, inplace = True)

# convert to np array for the NN Class
MNIST_test = np.array(MNIST_test)
MNIST_train = np.array(MNIST_train)
MNIST_test = MNIST_test.T
print(MNIST_test.shape)
X_test = MNIST_test[0:784]/255
Y_test = MNIST_test[784:795]
print(X_test.shape)
print(Y_test.shape)
MNIST_train = MNIST_train.T
X_train = MNIST_train[0:784]/255
Y_train = MNIST_train[784:795]


(794, 10000)
(784, 10000)
(10, 10000)


## Letter Recognition Data Set
Contains 17 features that describe the original images. 

In [168]:
Letter = pd.read_csv("letter-recognition.data",names=["lettr","x-box","y-box","width","high","onpix","x-bar","y-bar","x2bar","y2bar","xybar","x2ybr","xy2br","x-ege","xegvy","y-ege","yegvx"])
Letter = pd.concat([Letter,pd.get_dummies(Letter.lettr,prefix = "letter")],axis=1)
Letter.drop(columns=['lettr'], axis =1, inplace = True)

Letter_train, Letter_test = train_test_split(Letter,test_size=.2,random_state =5, shuffle=True)
Letter_train = np.array(Letter_train)
Letter_test = np.array(Letter_test)
Letter_train = Letter_train.T
Letter_test = Letter_test.T
L_X_test = Letter_test[0:16]
L_Y_test = Letter_test[16:]
L_X_train = Letter_train[0:16]
print(L_X_train.shape)
L_Y_train = Letter_train[16:]
print(L_Y_train.shape)


(16, 16000)
(26, 16000)


## Layers of NN
My Hidden and Output layers will both be fully connected for simplicity.
### Input Layer
Input layer # of nodes = # of features in the data set. For our data set there are 784 features representing each of the pixels
### Hidden Layers
I am going to choose to do a single hidden layer conisisting of ten nodes. I will use ReLU as the activation function for the hidden layer.
### Output Layers
Output layer # of nodes = 1 for regression, and for multiclass problems it is dependent on the number of classes you are trying to predict. In this case, I am trying to predict the numeral an image represents (0-9). Therefore, my output layer will have 10 nodes representing the digits 0-9. I will use ReLU as the activation function for the output layer.
![NN_Diagram](NN_Diagram.png)



## Training
Once the nodes and layers are setup, it is time to think about how we train the weights and biasis corresponding to each layer. This is a 3 part process. Weights and Biasis are called the learning parameters. 

### 1. Forward Propogation

### 2. Backwards Propogation

### 3. Parameter Updates

In [172]:
# class for NN with one hidden layer
class NN:
    def __init__(self,numInputNodes,numHiddenNodes,numOutputNodes,learningRate,iterations):
        self.numInputNodes = numInputNodes
        self.numHiddenNodes = numHiddenNodes
        self.numOutputNodes = numOutputNodes
        self.learningRate = learningRate
        self.iterations = iterations
    
    # initializes a weight and biases as np arrays
    def Init_Learning_Params(self):
        # gives a value between 0 and 1
        # want a value between -.5 and .5
        self.W1 = np.random.rand(self.numHiddenNodes,self.numInputNodes) -0.5
        self.B1 = np.random.rand(self.numHiddenNodes, 1)
        self.W2 = np.random.rand(self.numOutputNodes, self.numHiddenNodes) -0.5
        self.B2 = np.random.rand(self.numOutputNodes, 1)
        
        
        #return W1,W2,B1,B2
    
    # TODO make sure this works with np array
    def Relu(self,value):
        return np.maximum(value,0)
    # TODO make sure this works with np array
    def Relu_Prime(self,value):
        return value>0
    
    # performs forward propogation through the hidden layer and output layer
    # returns the result after being passed through the activation function ReLU
    # 
    def Forward_Propogate(self,Input):
        # finds the sum of the inputs to the hidden nodes from the input nodes
        self.Hidden = self.W1.dot(Input)+self.B1
        # runs relu on the sum array
        self.Hidden_Activated = self.Relu(self.Hidden)
        # uses hidden_activated as the input to multiply by the weights
        self.Output = self.W2.dot(self.Hidden_Activated)+self.B2
        self.Output_Activated = self.Relu(self.Output)
        
    def Forward_Propogate_Predict(self,Input):
        # finds the sum of the inputs to the hidden nodes from the input nodes
        Hidden = self.W1.dot(Input)+self.B1
        # runs relu on the sum array
        Hidden_Activated = self.Relu(Hidden)
        # uses hidden_activated as the input to multiply by the weights
        Output = self.W2.dot(Hidden_Activated)+self.B2
        Output_Activated = self.Relu(Output)
        return Hidden, Hidden_Activated, Output, Output_Activated
        
    def Backward_Propogation(self,Input, Expected_Output):
        # Output to Hidden layer
        
        Hidden_Prime = self.Hidden_Activated - Expected_Output
        self.W2_Prime = 1/ Expected_Output.size * Hidden_Prime.dot(self.Output_Activated.T)
        self.B2_Prime = 1/ Expected_Output.size * np.sum(Hidden_Prime)
        # Hidden layer to Input
        Output_Prime = self.W2.T.dot(Hidden_Prime)*self.Relu_Prime(self.Output)
        self.W1_Prime = 1/ Expected_Output.size * Hidden_Prime.dot(Input.T)
        self.B1_Prime = 1/Expected_Output.size * np.sum(Output_Prime)
    
    def Train(self,Input, Expected_Output):
        self.Init_Learning_Params()
        for x in range(self.iterations):
            self.Forward_Propogate(Input)
            self.Backward_Propogation(Input, Expected_Output)
            self.W1 = self.W1-self.learningRate * self.W1_Prime
            self.W2 = self.W2-self.learningRate * self.W2_Prime
            self.B1 = self.B1-self.learningRate * self.B1_Prime
            self.B2 = self.B2-self.learningRate * self.B2_Prime
            print("Iteration ", x)
            print("Accuracy = ", np.sum(np.argmax(self.Output_Activated,0)== Expected_Output)/Expected_Output.size)

    
    def Test(self,Input,Expected_Output):
        _,_,_,Output_Activated = self.Forward_Propogate_Predict(Input)
        print("Accuracy = ", np.sum(np.argmax(Output_Activated,0)== Expected_Output)/Expected_Output.size)
        
        
    
    

In [181]:
myNet = NN(784,10,10,0.15,400)
myNet.Train(X_train,Y_train)

(10, 60000)
Iteration  0
Accuracy =  0.00235
(10, 60000)
Iteration  1
Accuracy =  0.007825
(10, 60000)
Iteration  2
Accuracy =  0.011606666666666666
(10, 60000)
Iteration  3
Accuracy =  0.013266666666666666
(10, 60000)
Iteration  4
Accuracy =  0.01403
(10, 60000)
Iteration  5
Accuracy =  0.013986666666666666
(10, 60000)
Iteration  6
Accuracy =  0.01348
(10, 60000)
Iteration  7
Accuracy =  0.013243333333333333
(10, 60000)
Iteration  8
Accuracy =  0.01304
(10, 60000)
Iteration  9
Accuracy =  0.012528333333333334
(10, 60000)
Iteration  10
Accuracy =  0.011851666666666667
(10, 60000)
Iteration  11
Accuracy =  0.011358333333333333
(10, 60000)
Iteration  12
Accuracy =  0.010901666666666667
(10, 60000)
Iteration  13
Accuracy =  0.010656666666666667
(10, 60000)
Iteration  14
Accuracy =  0.01035
(10, 60000)
Iteration  15
Accuracy =  0.010118333333333333
(10, 60000)
Iteration  16
Accuracy =  0.00996
(10, 60000)
Iteration  17
Accuracy =  0.010005
(10, 60000)
Iteration  18
Accuracy =  0.0100683333

(10, 60000)
Iteration  151
Accuracy =  0.6278233333333333
(10, 60000)
Iteration  152
Accuracy =  0.6312683333333333
(10, 60000)
Iteration  153
Accuracy =  0.6345533333333333
(10, 60000)
Iteration  154
Accuracy =  0.6375983333333334
(10, 60000)
Iteration  155
Accuracy =  0.6402366666666667
(10, 60000)
Iteration  156
Accuracy =  0.6433716666666667
(10, 60000)
Iteration  157
Accuracy =  0.6461133333333333
(10, 60000)
Iteration  158
Accuracy =  0.6491283333333333
(10, 60000)
Iteration  159
Accuracy =  0.6516183333333333
(10, 60000)
Iteration  160
Accuracy =  0.6547216666666666
(10, 60000)
Iteration  161
Accuracy =  0.6571766666666666
(10, 60000)
Iteration  162
Accuracy =  0.6596966666666667
(10, 60000)
Iteration  163
Accuracy =  0.6625616666666667
(10, 60000)
Iteration  164
Accuracy =  0.6652916666666666
(10, 60000)
Iteration  165
Accuracy =  0.6680066666666666
(10, 60000)
Iteration  166
Accuracy =  0.6706
(10, 60000)
Iteration  167
Accuracy =  0.673345
(10, 60000)
Iteration  168
Accuracy 

(10, 60000)
Iteration  303
Accuracy =  0.852215
(10, 60000)
Iteration  304
Accuracy =  0.852785
(10, 60000)
Iteration  305
Accuracy =  0.853325
(10, 60000)
Iteration  306
Accuracy =  0.8538033333333334
(10, 60000)
Iteration  307
Accuracy =  0.8542683333333333
(10, 60000)
Iteration  308
Accuracy =  0.8547783333333333
(10, 60000)
Iteration  309
Accuracy =  0.8553633333333334
(10, 60000)
Iteration  310
Accuracy =  0.8558883333333334
(10, 60000)
Iteration  311
Accuracy =  0.8565033333333333
(10, 60000)
Iteration  312
Accuracy =  0.8571933333333334
(10, 60000)
Iteration  313
Accuracy =  0.8576433333333333
(10, 60000)
Iteration  314
Accuracy =  0.8580483333333333
(10, 60000)
Iteration  315
Accuracy =  0.8584383333333333
(10, 60000)
Iteration  316
Accuracy =  0.8589016666666667
(10, 60000)
Iteration  317
Accuracy =  0.85944
(10, 60000)
Iteration  318
Accuracy =  0.85989
(10, 60000)
Iteration  319
Accuracy =  0.860265
(10, 60000)
Iteration  320
Accuracy =  0.8607433333333333
(10, 60000)
Iterat

In [182]:
myNet.Test(X_test,Y_test)

Accuracy =  0.88794


In [175]:
myNet2 = NN(16,26,26,0.05,10000)
myNet2.Train(L_X_train,L_Y_train)

(26, 16000)
Iteration  0
Accuracy =  0.02670673076923077
(26, 16000)
Iteration  1
Accuracy =  0.042903846153846154
(26, 16000)
Iteration  2
Accuracy =  0.05146875
(26, 16000)
Iteration  3
Accuracy =  0.05397836538461538
(26, 16000)
Iteration  4
Accuracy =  0.05365865384615385
(26, 16000)
Iteration  5
Accuracy =  0.05194230769230769
(26, 16000)
Iteration  6
Accuracy =  0.05044230769230769
(26, 16000)
Iteration  7
Accuracy =  0.04982211538461538
(26, 16000)
Iteration  8
Accuracy =  0.04853846153846154
(26, 16000)
Iteration  9
Accuracy =  0.04576923076923077
(26, 16000)
Iteration  10
Accuracy =  0.04475
(26, 16000)
Iteration  11
Accuracy =  0.04397596153846154
(26, 16000)
Iteration  12
Accuracy =  0.04273317307692308
(26, 16000)
Iteration  13
Accuracy =  0.04145913461538461
(26, 16000)
Iteration  14
Accuracy =  0.04035576923076923
(26, 16000)
Iteration  15
Accuracy =  0.039788461538461536
(26, 16000)
Iteration  16
Accuracy =  0.038752403846153846
(26, 16000)
Iteration  17
Accuracy =  0.03

Accuracy =  0.020786057692307694
(26, 16000)
Iteration  147
Accuracy =  0.02084375
(26, 16000)
Iteration  148
Accuracy =  0.020841346153846155
(26, 16000)
Iteration  149
Accuracy =  0.02084375
(26, 16000)
Iteration  150
Accuracy =  0.020783653846153848
(26, 16000)
Iteration  151
Accuracy =  0.02090144230769231
(26, 16000)
Iteration  152
Accuracy =  0.02090144230769231
(26, 16000)
Iteration  153
Accuracy =  0.020961538461538462
(26, 16000)
Iteration  154
Accuracy =  0.021021634615384616
(26, 16000)
Iteration  155
Accuracy =  0.021079326923076923
(26, 16000)
Iteration  156
Accuracy =  0.021016826923076923
(26, 16000)
Iteration  157
Accuracy =  0.02107451923076923
(26, 16000)
Iteration  158
Accuracy =  0.021072115384615384
(26, 16000)
Iteration  159
Accuracy =  0.021134615384615384
(26, 16000)
Iteration  160
Accuracy =  0.021060096153846155
(26, 16000)
Iteration  161
Accuracy =  0.021180288461538462
(26, 16000)
Iteration  162
Accuracy =  0.021295673076923077
(26, 16000)
Iteration  163
Acc

Iteration  294
Accuracy =  0.022963942307692307
(26, 16000)
Iteration  295
Accuracy =  0.022966346153846153
(26, 16000)
Iteration  296
Accuracy =  0.022963942307692307
(26, 16000)
Iteration  297
Accuracy =  0.02295673076923077
(26, 16000)
Iteration  298
Accuracy =  0.02306971153846154
(26, 16000)
Iteration  299
Accuracy =  0.02306971153846154
(26, 16000)
Iteration  300
Accuracy =  0.023067307692307693
(26, 16000)
Iteration  301
Accuracy =  0.023129807692307693
(26, 16000)
Iteration  302
Accuracy =  0.023127403846153846
(26, 16000)
Iteration  303
Accuracy =  0.023067307692307693
(26, 16000)
Iteration  304
Accuracy =  0.02288221153846154
(26, 16000)
Iteration  305
Accuracy =  0.022942307692307692
(26, 16000)
Iteration  306
Accuracy =  0.023185096153846153
(26, 16000)
Iteration  307
Accuracy =  0.023182692307692307
(26, 16000)
Iteration  308
Accuracy =  0.023185096153846153
(26, 16000)
Iteration  309
Accuracy =  0.023129807692307693
(26, 16000)
Iteration  310
Accuracy =  0.023137019230769

Accuracy =  0.02588701923076923
(26, 16000)
Iteration  442
Accuracy =  0.02588701923076923
(26, 16000)
Iteration  443
Accuracy =  0.02588701923076923
(26, 16000)
Iteration  444
Accuracy =  0.025889423076923077
(26, 16000)
Iteration  445
Accuracy =  0.026007211538461538
(26, 16000)
Iteration  446
Accuracy =  0.026007211538461538
(26, 16000)
Iteration  447
Accuracy =  0.02594951923076923
(26, 16000)
Iteration  448
Accuracy =  0.025764423076923077
(26, 16000)
Iteration  449
Accuracy =  0.025819711538461538
(26, 16000)
Iteration  450
Accuracy =  0.025639423076923077
(26, 16000)
Iteration  451
Accuracy =  0.02569951923076923
(26, 16000)
Iteration  452
Accuracy =  0.025697115384615384
(26, 16000)
Iteration  453
Accuracy =  0.02569951923076923
(26, 16000)
Iteration  454
Accuracy =  0.025872596153846152
(26, 16000)
Iteration  455
Accuracy =  0.025877403846153845
(26, 16000)
Iteration  456
Accuracy =  0.025814903846153845
(26, 16000)
Iteration  457
Accuracy =  0.025819711538461538
(26, 16000)
I

(26, 16000)
Iteration  588
Accuracy =  0.02601923076923077
(26, 16000)
Iteration  589
Accuracy =  0.025899038461538463
(26, 16000)
Iteration  590
Accuracy =  0.025899038461538463
(26, 16000)
Iteration  591
Accuracy =  0.02588701923076923
(26, 16000)
Iteration  592
Accuracy =  0.02588701923076923
(26, 16000)
Iteration  593
Accuracy =  0.02582451923076923
(26, 16000)
Iteration  594
Accuracy =  0.025882211538461538
(26, 16000)
Iteration  595
Accuracy =  0.025641826923076923
(26, 16000)
Iteration  596
Accuracy =  0.025822115384615384
(26, 16000)
Iteration  597
Accuracy =  0.025882211538461538
(26, 16000)
Iteration  598
Accuracy =  0.025882211538461538
(26, 16000)
Iteration  599
Accuracy =  0.02587980769230769
(26, 16000)
Iteration  600
Accuracy =  0.02594230769230769
(26, 16000)
Iteration  601
Accuracy =  0.025882211538461538
(26, 16000)
Iteration  602
Accuracy =  0.025944711538461538
(26, 16000)
Iteration  603
Accuracy =  0.02600480769230769
(26, 16000)
Iteration  604
Accuracy =  0.026004

...

(26, 16000)
Iteration  9268
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9269
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9270
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9271
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9272
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9273
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9274
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9275
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9276
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9277
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9278
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9279
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9280
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9281
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9282
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9283
Accuracy =  0.05813461538461538
(26, 16000)
Iteration  9284
Accuracy =  

(26, 16000)
Iteration  9413
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9414
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9415
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9416
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9417
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9418
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9419
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9420
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9421
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9422
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9423
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9424
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9425
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9426
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9427
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9428
Accuracy =  0.05825240384615385
(26, 16000)
Iteration  9429
Accuracy =  

(26, 16000)
Iteration  9561
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9562
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9563
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9564
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9565
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9566
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9567
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9568
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9569
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9570
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9571
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9572
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9573
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9574
Accuracy =  0.058610576923076925
(26, 16000)
Iteration  9575
Accuracy =  0.05867067307692308
(26, 16000)
Iteration  9576
Accuracy =  0.05867067307692308
(26, 16000)
Iteration  957

Iteration  9711
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9712
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9713
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9714
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9715
Accuracy =  0.05920432692307692
(26, 16000)
Iteration  9716
Accuracy =  0.05920432692307692
(26, 16000)
Iteration  9717
Accuracy =  0.05920432692307692
(26, 16000)
Iteration  9718
Accuracy =  0.05920432692307692
(26, 16000)
Iteration  9719
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9720
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9721
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9722
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9723
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9724
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9725
Accuracy =  0.059264423076923076
(26, 16000)
Iteration  9726
Accuracy =  0.05932451923076923
(26, 16000)
Iteration  9727
Accuracy =  0

Iteration  9854
Accuracy =  0.05998076923076923
(26, 16000)
Iteration  9855
Accuracy =  0.05998076923076923
(26, 16000)
Iteration  9856
Accuracy =  0.05998076923076923
(26, 16000)
Iteration  9857
Accuracy =  0.05998076923076923
(26, 16000)
Iteration  9858
Accuracy =  0.05998076923076923
(26, 16000)
Iteration  9859
Accuracy =  0.05998076923076923
(26, 16000)
Iteration  9860
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9861
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9862
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9863
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9864
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9865
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9866
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9867
Accuracy =  0.06004086538461539
(26, 16000)
Iteration  9868
Accuracy =  0.06010096153846154
(26, 16000)
Iteration  9869
Accuracy =  0.06010096153846154
(26, 16000)
Iteration  9870
Accuracy =  0.0601009615

In [178]:
myNet2.Test(L_X_test,L_Y_test)

Accuracy =  0.06121153846153846


## Results
The nueral network trained on the MNIST data set performed very well with an accuracy of approximately 88% on the testing data. The neural network trained on the Letter Recognition Data set on the other hand did not perform so well with accuraccy of approximately 6%. This may have been due to a failure to generalize enough in my neural network to allow for more options in specifying the size and number of hidden layers. It may also be due to the fact that I only used relu as an activation function.